In [392]:
import numpy as np
import matplotlib.pyplot as plt
from polar_codes.polar_code import PolarCode
from polar_codes.channels.bpsk_awgn_channel import BpskAwgnChannel

In [393]:
ARIKAN_KERNEL = np.array([[1,0],
                         [1,1 ]], np.int8)

In [394]:
# BIG_BOY
kernel = np.array([
    [1, 0, 0, 0],
    [1, 1, 0, 0],
    [0, 1, 1, 0],
    [1, 1, 1, 1]])

In [395]:
# GEN MATRIX FOR BIG_BOY
def gen_mtrx(deg , k=kernel):    
    """Generator matrix of the polar code"""
    result = kernel
    for i in range(deg-1):
        result = np.kron(result, k)
    return result

In [396]:
# ЛОВИ ЯДРО
oder = 4
G = gen_mtrx(oder)
G.shape

(256, 256)

In [404]:
interfer_num = np.zeros(G.shape[1],"int")
for i, j in enumerate(G):
    interfer_num[i] = np.sum(j)

mask = np.ones_like(interfer_num)

for i, j in enumerate(interfer_num):
    if j > np.mean(interfer_num):
        mask[i] = 0

print(np.sum(mask)/mask.size)
print(interfer_num)
print(mask)
print(interfer_num*mask)

0.63671875
[  1   2   2   4   2   4   4   8   2   4   4   8   4   8   8  16   2   4
   4   8   4   8   8  16   4   8   8  16   8  16  16  32   2   4   4   8
   4   8   8  16   4   8   8  16   8  16  16  32   4   8   8  16   8  16
  16  32   8  16  16  32  16  32  32  64   2   4   4   8   4   8   8  16
   4   8   8  16   8  16  16  32   4   8   8  16   8  16  16  32   8  16
  16  32  16  32  32  64   4   8   8  16   8  16  16  32   8  16  16  32
  16  32  32  64   8  16  16  32  16  32  32  64  16  32  32  64  32  64
  64 128   2   4   4   8   4   8   8  16   4   8   8  16   8  16  16  32
   4   8   8  16   8  16  16  32   8  16  16  32  16  32  32  64   4   8
   8  16   8  16  16  32   8  16  16  32  16  32  32  64   8  16  16  32
  16  32  32  64  16  32  32  64  32  64  64 128   4   8   8  16   8  16
  16  32   8  16  16  32  16  32  32  64   8  16  16  32  16  32  32  64
  16  32  32  64  32  64  64 128   8  16  16  32  16  32  32  64  16  32
  32  64  32  64  64 128  16  32  32  64

In [403]:
256*0.6367185-128

34.99993599999999

In [398]:
def kernel(deg):    
    """Generator matrix of the polar code"""
    result = ARIKAN_KERNEL
    for i in range(deg-1):
        result = np.kron(result, ARIKAN_KERNEL)
    return result

In [399]:
def encoding(data_en, G_matrix):
    """Encoding function"""
    return np.dot(data_en, G_matrix) % 2

In [400]:
def ber_calculation(sent_mes, received_mes):
    num_errors = 0
    for i in range(len(sent_mes)):
        if sent_mes[i] != received_mes[i]:
            num_errors += 1
    return num_errors / len(sent_mes)

In [401]:
def bpsk_modulation(signal):
    modulated_signal = []
    for i in signal:
        modulated_signal.append(np.where(i > 0, 1, -1))
    return modulated_signal

def bpsk_demodulation(signal):
    demodulated_signal = []
    for i in signal:
        demodulated_signal.append(np.where(i > 0, 1, 0))
    return demodulated_signal

In [402]:
def awgn_generator(snr, signal_len):
    return np.random.randn(signal_len)*(1 / np.sqrt(2 * 10 ** (snr / 10)))